In [2]:
from pydub import AudioSegment
import numpy as np

In [29]:
def normalize_min_max(arr):
    min_val = np.min(arr)
    max_val = np.max(arr)
    return (arr - min_val) / (max_val - min_val)

def load_original():
    a = AudioSegment.from_mp3("/home/julixquid/Downloads/ml (2)/ml/original.mp3")
    b = a.set_frame_rate(44100).set_channels(1)
    y = np.array(b.get_array_of_samples())
    y = normalize_min_max(y)
    y = np.pad(y, (0, 80000 - len(y)), mode='constant', constant_values=0)
    return y[:-1].astype(np.float32).reshape(1, 1, -1)


In [23]:
a = AudioSegment.from_mp3("/home/julixquid/Downloads/ml (2)/ml/original.mp3")


In [24]:
b = a.set_frame_rate(44100).set_channels(1)

In [25]:
y = np.array(b.get_array_of_samples())

In [26]:
y = normalize_min_max(y)

/tmp/ipykernel_33171/2004056986.py:4: RuntimeWarning: overflow encountered in scalar subtract
  return (arr - min_val) / (max_val - min_val)


In [27]:
y = np.pad(y, (0, 80000 - len(y)), mode='constant', constant_values=0)

In [32]:
arr = load_original()

/tmp/ipykernel_33171/3974179784.py:4: RuntimeWarning: overflow encountered in scalar subtract
  return (arr - min_val) / (max_val - min_val)


In [37]:
flattened_arr = best_input.flatten()

# Step 2: Convert to Python list and format as string
array_str = "[" + ", ".join(map(str, flattened_arr.tolist())) + "]"

# Step 3: Save to text file
with open("array_data.txt", "w") as f:
    f.write(array_str)

print("Array saved to array_data.txt")

Array saved to array_data.txt


In [45]:
import numpy as np
import torch
import torch.nn as nn

# --- Define the neural network (same as in Flask app) ---
class Conv1DNet(nn.Module):
    def __init__(self, num_classes):
        super(Conv1DNet, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3)
        self.pool2 = nn.MaxPool1d(2)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3)
        self.global_pool = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.relu(self.conv3(x))
        x = self.global_pool(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# --- Load the pretrained model ---
model = Conv1DNet(num_classes=2)
model.load_state_dict(tor	KalmarCTF 2025	209.0000	0.983ch.load('/home/julixquid/Downloads/ml (2)/ml/model_weights.pth'))#, map_location='cpu'))
model.eval()  # Disable dropout/batchnorm

Conv1DNet(
  (conv1): Conv1d(1, 16, kernel_size=(3,), stride=(1,))
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (global_pool): AdaptiveMaxPool1d(output_size=1)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
  (relu): ReLU()
)

In [78]:
# --- Genetic Algorithm Parameters ---
POPULATION_SIZE = 25  # Number of candidates
MUTATION_RATE = 0.0007 # Probability of mutation per element
NUM_GENERATIONS = 50000  # Max iterations

# --- Fitness Function ---
def evaluate_fitness(x):
    x_tensor = torch.tensor(x, dtype=torch.float32).view(1, 1, -1)
    with torch.no_grad():
        output = model(x_tensor)
        prob_class_1 = torch.softmax(output, dim=1)[0, 1].item()
    return prob_class_1  # Higher = better

# --- Initialize Population ---
population = [best_input]

# --- Evolution Loop ---
for generation in range(NUM_GENERATIONS):
    # 1. Evaluate fitness
    fitness = [evaluate_fitness(x) for x in population]
    best_idx = np.argmax(fitness)
    best_fitness = fitness[best_idx]
    
    #print(f"Generation {generation}: Best P(class=1) = {best_fitness:.17f}")
    
    # Exit if we fool the network
    if best_fitness > 0.99:
        print("Success! Found adversarial input.")
        break
    
    # 2. Select top candidates (elite selection)
    elite_indices = np.argsort(fitness)[-2:]  # Top 10
    elites = [population[i] for i in elite_indices]
    
    # 3. Breed new population
    new_population = elites.copy()  # Keep elites
    
    while len(new_population) < POPULATION_SIZE:
        # Pick a random elite parent
        parent = elites[np.random.randint(len(elites))]
        
        # Create a child with mutation
        child = parent.copy()
        mask = np.random.rand(79999) < MUTATION_RATE
        child[mask] = np.clip(child[mask] + np.random.normal(0, 0.07, size=np.sum(mask)), 0, 1)
        
        new_population.append(child)
    
    population = new_population

# --- Save the Best Input ---
    best_input = population[best_idx]
    if (generation % 50)==0:
        np.savetxt('adversarial_input.txt', best_input, fmt='%.17f', delimiter=',')
        print(f"Generation {generation}: Best P(class=1) = {best_fitness:.17f}")
print("Saved adversarial input to 'adversarial_input.txt'")

Generation 0: Best P(class=1) = 0.98325127363204956
Generation 50: Best P(class=1) = 0.98377984762191772
Generation 100: Best P(class=1) = 0.98412895202636719
Generation 150: Best P(class=1) = 0.98425227403640747
Generation 200: Best P(class=1) = 0.98431360721588135
Generation 250: Best P(class=1) = 0.98435312509536743
Generation 300: Best P(class=1) = 0.98438489437103271
Generation 350: Best P(class=1) = 0.98441547155380249
Generation 400: Best P(class=1) = 0.98444181680679321
Generation 450: Best P(class=1) = 0.98446214199066162
Generation 500: Best P(class=1) = 0.98447477817535400
Generation 550: Best P(class=1) = 0.98450583219528198
Generation 600: Best P(class=1) = 0.98451578617095947
Generation 650: Best P(class=1) = 0.98452579975128174
Generation 700: Best P(class=1) = 0.98454165458679199
Generation 750: Best P(class=1) = 0.98454916477203369
Generation 800: Best P(class=1) = 0.98456788063049316
Generation 850: Best P(class=1) = 0.98457700014114380
Generation 900: Best P(class=1)

KeyboardInterrupt: 

In [36]:
best_input

array([0.34411166, 0.12414883, 0.53693168, ..., 0.20192388, 0.58226629,
       0.7813683 ], shape=(79999,))